In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Rescaling
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from shutil import copyfile


import os
import shutil
import cv2
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec

from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
emotions = ['angry', 'happy', 'relaxed', 'sad']
dataset_path = '/kaggle/input/dog-emotion/Dog Emotion'

In [3]:
class_names = sorted(os.listdir(dataset_path))
class_names.remove('labels.csv')

images = []
labels = []

labels_df = pd.read_csv(os.path.join(dataset_path, 'labels.csv'), index_col=None)
labels_df = labels_df.sort_values(by=['label', 'filename'])
print(labels_df.head())

for _, filename, label in labels_df.iloc:
    image = cv2.imread(os.path.join(dataset_path, label, filename))
    image = cv2.resize(image, (192, 192))
    image = np.array(image)[:, :, ::-1]
    images.append(image)
    
    encode_label = np.zeros(4)
    encode_label[class_names.index(label)] = 1
    labels.append(encode_label)

labels = np.asarray(labels)
images = np.asarray(images)

print(f'\nlabels shape: {labels.shape}\nimages shape: {images.shape}')


      Unnamed: 0                               filename  label
2017        2017  09dUVMcjCDfOtbeYDQg5Fvu3GPHWJg811.jpg  angry
553          553   0AvKtuzA7LfxnKaO0bey9mQMLnxXad73.jpg  angry
978          978  0C5yo7GxMy8lztxNZvSdfEx2gSPRTR701.jpg  angry
946          946  0C9jEgFQHsh36W5U2u5CA98lB7C5eX806.jpg  angry
82            82  0RXraPIKC00Dz1qkuMbj8XbuR80g5Z893.jpg  angry

labels shape: (4000, 4)
images shape: (4000, 192, 192, 3)


In [4]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.1, random_state=24)

In [5]:
def show_images(generator, nRows = 1, nCols = 2, WidthSpace = 0.02, HeightSpace = 0.15):
    """
    Plots random num_images images from generator

    Args:
        generator: a generator instance
        nRows: number of rows
        nCols: number of columns
        WidthSpace: width space
        HeightSpace: height space
    """
    global emotions
    gs = gridspec.GridSpec(nRows, nCols)     
    gs.update(wspace=WidthSpace, hspace=HeightSpace)
    
    images, labels = generator.next()
    labels = labels.astype('int32')
    num_images = nRows * nCols
    
    
    plt.figure(figsize=(20,10))
    
    for i, (image, label) in enumerate(zip(images, labels)):
        plt.subplot(gs[i])
        plt.imshow(image)
#         index_label = label_to_index(str(label))
        plt.title(emotions[label])
        if i == num_images - 1:
            break
    plt.show()
        

In [6]:
# show_images(train_generator, 2, 3)

In [7]:
model = Sequential()

# Inputs and rescaling
model.add(Rescaling(scale=1./255, input_shape=(192, 192, 3)))

# Convolutional block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))

# Convolutional block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))

# Convolutional layres 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))

# Convolutional block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())

# Dense block
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 192, 192, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 192, 192, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 192, 192, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 96, 96, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 96, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 96, 96, 128)       147584    
                                                        

In [8]:
def scheduler(epoch, learning_rate):
    if epoch < 20:
        return learning_rate
    else:
        return learning_rate * tf.math.exp(-0.2)

callback = LearningRateScheduler(scheduler)

In [9]:
history = model.fit(images, labels,    
                    validation_data = (X_val, y_val),
                    epochs=25, callbacks=[callback], verbose=1)

Epoch 1/25
125/125 [==============================] - 53s 232ms/step - loss: 1.3854 - accuracy: 0.2430 - val_loss: 1.3639 - val_accuracy: 0.2425 - lr: 1.0000e-04
Epoch 2/25
125/125 [==============================] - 27s 217ms/step - loss: 1.3005 - accuracy: 0.3330 - val_loss: 1.2605 - val_accuracy: 0.3675 - lr: 1.0000e-04
Epoch 3/25
125/125 [==============================] - 28s 225ms/step - loss: 1.2556 - accuracy: 0.3675 - val_loss: 1.2259 - val_accuracy: 0.4025 - lr: 1.0000e-04
Epoch 4/25
125/125 [==============================] - 29s 228ms/step - loss: 1.2255 - accuracy: 0.3930 - val_loss: 1.1782 - val_accuracy: 0.4475 - lr: 1.0000e-04
Epoch 5/25
125/125 [==============================] - 28s 225ms/step - loss: 1.1772 - accuracy: 0.4300 - val_loss: 1.0877 - val_accuracy: 0.4900 - lr: 1.0000e-04
Epoch 6/25
125/125 [==============================] - 28s 228ms/step - loss: 1.0874 - accuracy: 0.4952 - val_loss: 0.9484 - val_accuracy: 0.6025 - lr: 1.0000e-04
Epoch 7/25
125/125 [========